# Baltimore Crime Interactive Data Map

<h2>Part 1: Import and Set Up the Dataset</h2>

In [1]:
!pip install folium
import folium
from folium.map import *
import requests
import pandas

arrest_table = pandas.read_csv("http://www.hcbravo.org/IntroDataSci/misc/BPD_Arrests.csv")

arrest_table["race_new"] = arrest_table["sex"]
arrest_table["sex_new"] = arrest_table["race"]
arrest_table["race"] = arrest_table["race_new"]
arrest_table["sex"] = arrest_table["sex_new"]
arrest_table = arrest_table.drop('race_new', 1)
arrest_table = arrest_table.drop('sex_new', 1)

arrest_table = arrest_table[pandas.notnull(arrest_table["Location 1"])]

arrest_table["lat"], arrest_table["long"] = arrest_table["Location 1"].str.split(",").str
arrest_table["lat"] = arrest_table["lat"].str.replace("(", "").astype(float)
arrest_table["long"] = arrest_table["long"].str.replace(")", "").astype(float)

arrest_table.head()

    100% |████████████████████████████████| 81kB 1.1MB/s ta 0:00:01
  Running setup.py bdist_wheel for folium ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/04/d0/a0/b2b8356443364ae79743fce0b9b6a5b045f7560742129fde22
Successfully built folium


,arrest,age,sex,race,arrestDate,arrestTime,arrestLocation,incidentOffense,incidentLocation,charge,chargeDescription,district,post,neighborhood,Location 1,lat,long
1,11127013.0,37,M,B,01/01/2011,00:01,2000 Wilkens Ave,79-Other,Wilkens Av & S Payson St,1 1425,Reckless Endangerment || Hand Gun Violation,SOUTHERN,934.0,Carrollton Ridge,"(39.2814026274, -76.6483635135)",39.281403,-76.648364
2,11126887.0,46,M,B,01/01/2011,00:01,2800 Mayfield Ave,Unknown Offense,NaN,NaN,Unknown Charge,NORTHEASTERN,415.0,Belair-Edison,"(39.3227699160, -76.5735750473)",39.322770,-76.573575
3,11126873.0,50,M,B,01/01/2011,00:04,2100 Ashburton St,79-Other,2100 Ashburton St,1 1106,Reg Firearm:Illegal Possession || Hgv,WESTERN,735.0,Panway/Braddish Avenue,"(39.3117196723, -76.6623546313)",39.311720,-76.662355
4,11126968.0,33,M,B,01/01/2011,00:05,4000 Wilsby Ave,Unknown Offense,1700 Aliceanna St,NaN,Unknown Charge,NORTHERN,525.0,Pen Lucy,"(39.3382885254, -76.6045667070)",39.338289,-76.604567
5,11127041.0,41,M,B,01/01/2011,00:05,2900 Spellman Rd,81-Recovered Property,2900 Spelman Rd,1 1425,Reckless Endangerment || Handgun Violation,SOUTHERN,924.0,Cherry Hill,"(39.2449886230, -76.6273582432)",39.244989,-76.627358


<h2>Part 2: Generate the Interactive Map Using the Dataset</h2>
<p>This map illustrates the sample of 250 random different arrest cases happened in Baltimore. In particular, this map pinpoints the locations where these arrest happened using the given coordinates in the dataset. For each pinpoint, it includes the basic info of the arrest such as the arrest number, arrest location, and age of the arrested criminal. Moreover, the circles on the map indicates the race of the arrested criminals. In addition, each pinpoint is illustrated by a marker with different color to represent the sex of the criminal which also contains the basic information as users click on them.</p>

In [9]:
import random
map_osm = folium.Map(location=[39.29, -76.61], zoom_start=11)
myList = []
count = 0

# Add layers
arrests_sex = FeatureGroup(name='Sex')
arrests_race = FeatureGroup(name='Race')

# Generate Random Sample
while count < 250:
    aNum = random.randint(0, len(arrest_table['race']))
    if aNum not in myList:
        count += 1
        myList.append(aNum)
        
# Markers for arrest location
# Blue Markers: Male
# Red Markers : Female
for i in myList:
    if arrest_table['sex'].values[i] == 'M':
        arrests_sex.add_child(folium.Marker(
            location=[arrest_table['lat'].values[i],arrest_table['long'].values[i]],
            popup="Arrest #: " + str(arrest_table['arrest'].values[i]) + ". Arrest Location: " + arrest_table['arrestLocation'].values[i] + ". Age: " + str(arrest_table['age'].values[i]),
            icon=folium.Icon(color='blue'),
        ))
    else:
        arrests_sex.add_child(folium.Marker(
            location=[arrest_table['lat'].values[i],arrest_table['long'].values[i]],
            popup="Arrest #: " + str(arrest_table['arrest'].values[i]) + ". Arrest Location: " + arrest_table['arrestLocation'].values[i] + ". Age: " + str(arrest_table['age'].values[i]),
            icon=folium.Icon(color='red'),
        ))
        

# Circles for sex
# B - Black
# W - White
# A - Yellow
# U - Purple
# I - Orange
# H - Green
# NaN - No Circle
for i in myList:
    if arrest_table['race'].values[i] == 'B':
        arrests_race.add_child(folium.Circle(
            radius=100,
            location=[arrest_table['lat'].values[i],arrest_table['long'].values[i]],
            popup="Race : " + str(arrest_table['race'].values[i]),
            color='black',
            fill=True,
        ))
    elif arrest_table['race'].values[i] == 'W':
        arrests_race.add_child(folium.Circle(
            radius=100,
            location=[arrest_table['lat'].values[i],arrest_table['long'].values[i]],
            popup="Race : " + str(arrest_table['race'].values[i]),
            color='white',
            fill=True,
        ))
    elif arrest_table['race'].values[i] == 'A':
        arrests_race.add_child(folium.Circle(
            radius=100,
            location=[arrest_table['lat'].values[i],arrest_table['long'].values[i]],
            popup="Race : " + str(arrest_table['race'].values[i]),
            color='yellow',
            fill=True,
        ))
    elif arrest_table['race'].values[i] == 'U':
        arrests_sex.add_child(folium.Circle(
            radius=100,
            location=[arrest_table['lat'].values[i],arrest_table['long'].values[i]],
            popup="Race : " + str(arrest_table['race'].values[i]),
            color='purple',
            fill=True,
        ))
    elif arrest_table['race'].values[i] == 'H':
        arrests_race.add_child(folium.Circle(
            radius=100,
            location=[arrest_table['lat'].values[i],arrest_table['long'].values[i]],
            popup="Race : " + str(arrest_table['race'].values[i]),
            color='green',
            fill=True,
        ))
    elif arrest_table['race'].values[i] == 'I':
        arrests_race.add_child(folium.Circle(
            radius=100,
            location=[arrest_table['lat'].values[i],arrest_table['long'].values[i]],
            popup="Race : " + str(arrest_table['race'].values[i]),
            color='orange',
            fill=True,
        ))
        
map_osm.add_child(arrests_race)
map_osm.add_child(arrests_sex)
map_osm.add_child(folium.map.LayerControl())

map_osm

<h2>Insight</h2>
<p>The map clearly shows that most arrest cases in Baltimore happened around the center of the city. For all pinpoints, arrested criminals mostly happened to be male instead of female. Also, in this map, most criminals happened to be Black varies in different ages and multiple charges.</p>